In [1]:
import numpy as np
import pandas as pd
import cufflinks as cf
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot 
import plotly.express as px
from datetime import datetime,date

%matplotlib inline
init_notebook_mode(connected=True)
cf.set_config_file(sharing='private',theme='pearl',offline=True)
#print("Cufflinks Version : {}".format(cf.__version__))

df = pd.read_csv('./data/btc-usd-max.csv',index_col=0,parse_dates=True )
df = df.resample('D').first()
df['price'].fillna(method='ffill', inplace=True)
# to get the log price
# df['price'] = np.log(df['price'])
df2 = pd.read_csv('./data/eth-usd-max.csv',index_col=0,parse_dates=True )
df2 = df2.resample('D').first()
df2['price'].fillna(method='ffill', inplace=True)
# to get the log price
# df['price'] = np.log(df['price'])
df2.head()

,price,market_cap,total_volume
snapped_at,,,
2015-08-07 00:00:00+00:00,2.831620,0.000000e+00,9.062200e+04
2015-08-08 00:00:00+00:00,1.330750,8.033948e+07,3.680700e+05
2015-08-09 00:00:00+00:00,1.330750,NaN,NaN
2015-08-10 00:00:00+00:00,0.687586,4.155631e+07,4.004641e+05
2015-08-11 00:00:00+00:00,1.067379,6.453901e+07,1.518998e+06


In [2]:
from pmdarima import auto_arima

# Ignore harmless warnings
import warnings
warnings.filterwarnings("ignore")

In [3]:
# run auto_arima to evaluate p,d and q with defauls parameters
auto_arima(df2['price'])

ARIMA(order=(1, 1, 1), scoring_args={}, suppress_warnings=True,
      with_intercept=False)

In [4]:
auto_arima(df['price'],error_action='ignore').summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                               SARIMAX Results                                
==============================================================================
Dep. Variable:                      y   No. Observations:                 3434
Model:               SARIMAX(0, 1, 0)   Log Likelihood              -27572.971
Date:                Wed, 19 Oct 2022   AIC                          55147.941
Time:                        11:08:07   BIC                          55154.082
Sample:                    04-28-2013   HQIC                         55150.135
                         - 09-21-2022                                         
Covariance Type:                  opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
sigma2      5.542e+05   4025.186    137.688      0.000    5.46e+05    5.62e+05
===================================================================================
Ljung-Box (L1) (Q):                   2.19   Jarque-Bera (JB):             57706.99
Prob(Q):                              0.14   Prob(JB):                         0.00
Heteroskedasticity (H):            3470.58   Skew:                            -0.16
Prob(H) (two-sided):                  0.00   Kurtosis:                        23.08
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

In [5]:
# the more manual way of doing it 
stepwise_fit = auto_arima(df['price'], start_p=0, start_q=0,
                          max_p=30, max_q=30,
                          seasonal=False,
                          d=None, trace=True,
                          error_action='ignore',   # we don't want to know if an order does not work
                          suppress_warnings=True,  # we don't want convergence warnings
                          stepwise=True)           # set to stepwise

stepwise_fit.summary()

Performing stepwise search to minimize aic
 ARIMA(0,1,0)(0,0,0)[0] intercept   : AIC=55149.757, Time=0.11 sec
 ARIMA(1,1,0)(0,0,0)[0] intercept   : AIC=55149.564, Time=0.17 sec
 ARIMA(0,1,1)(0,0,0)[0] intercept   : AIC=55149.549, Time=0.39 sec
 ARIMA(0,1,0)(0,0,0)[0]             : AIC=55147.941, Time=0.09 sec
 ARIMA(1,1,1)(0,0,0)[0] intercept   : AIC=55151.542, Time=0.64 sec

Best model:  ARIMA(0,1,0)(0,0,0)[0]          
Total fit time: 1.409 seconds


<class 'statsmodels.iolib.summary.Summary'>
"""
                               SARIMAX Results                                
==============================================================================
Dep. Variable:                      y   No. Observations:                 3434
Model:               SARIMAX(0, 1, 0)   Log Likelihood              -27572.971
Date:                Wed, 19 Oct 2022   AIC                          55147.941
Time:                        11:08:08   BIC                          55154.082
Sample:                    04-28-2013   HQIC                         55150.135
                         - 09-21-2022                                         
Covariance Type:                  opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
sigma2      5.542e+05   4025.186    137.688      0.000    5.46e+05    5.62e+05
===================================================================================
Ljung-Box (L1) (Q):                   2.19   Jarque-Bera (JB):             57706.99
Prob(Q):                              0.14   Prob(JB):                         0.00
Heteroskedasticity (H):            3470.58   Skew:                            -0.16
Prob(H) (two-sided):                  0.00   Kurtosis:                        23.08
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

In [6]:
stepwise_fit = auto_arima(df['price'], start_p=0, start_q=0,
                          max_p=30, max_q=30,
                          start_P=0, seasonal=True,
                          d=None, D=1, trace=True,
                          error_action='ignore',   # we don't want to know if an order does not work
                          suppress_warnings=True,  # we don't want convergence warnings
                          stepwise=True)           # set to stepwise

stepwise_fit.summary()

Performing stepwise search to minimize aic
 ARIMA(0,1,0)(0,0,0)[0] intercept   : AIC=55149.757, Time=0.14 sec
 ARIMA(1,1,0)(0,0,0)[0] intercept   : AIC=55149.564, Time=0.24 sec
 ARIMA(0,1,1)(0,0,0)[0] intercept   : AIC=55149.549, Time=0.42 sec
 ARIMA(0,1,0)(0,0,0)[0]             : AIC=55147.941, Time=0.10 sec
 ARIMA(1,1,1)(0,0,0)[0] intercept   : AIC=55151.542, Time=0.73 sec

Best model:  ARIMA(0,1,0)(0,0,0)[0]          
Total fit time: 1.627 seconds


<class 'statsmodels.iolib.summary.Summary'>
"""
                               SARIMAX Results                                
==============================================================================
Dep. Variable:                      y   No. Observations:                 3434
Model:               SARIMAX(0, 1, 0)   Log Likelihood              -27572.971
Date:                Wed, 19 Oct 2022   AIC                          55147.941
Time:                        11:08:10   BIC                          55154.082
Sample:                    04-28-2013   HQIC                         55150.135
                         - 09-21-2022                                         
Covariance Type:                  opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
sigma2      5.542e+05   4025.186    137.688      0.000    5.46e+05    5.62e+05
===================================================================================
Ljung-Box (L1) (Q):                   2.19   Jarque-Bera (JB):             57706.99
Prob(Q):                              0.14   Prob(JB):                         0.00
Heteroskedasticity (H):            3470.58   Skew:                            -0.16
Prob(H) (two-sided):                  0.00   Kurtosis:                        23.08
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

In [7]:
stepwise_fit = auto_arima(df2['price'], start_p=0, start_q=0,
                          max_p=6, max_q=6,
                          start_P=0, seasonal=True,
                          d=None, D=1, trace=True,
                          error_action='ignore',   # we don't want to know if an order does not work
                          suppress_warnings=True,  # we don't want convergence warnings
                          stepwise=True)           # set to stepwise

stepwise_fit.summary()

Performing stepwise search to minimize aic
 ARIMA(0,1,0)(0,0,0)[0] intercept   : AIC=29216.358, Time=0.13 sec
 ARIMA(1,1,0)(0,0,0)[0] intercept   : AIC=29206.849, Time=0.27 sec
 ARIMA(0,1,1)(0,0,0)[0] intercept   : AIC=29207.592, Time=0.43 sec
 ARIMA(0,1,0)(0,0,0)[0]             : AIC=29214.511, Time=0.08 sec
 ARIMA(2,1,0)(0,0,0)[0] intercept   : AIC=29206.356, Time=0.47 sec
 ARIMA(3,1,0)(0,0,0)[0] intercept   : AIC=29207.335, Time=0.59 sec
 ARIMA(2,1,1)(0,0,0)[0] intercept   : AIC=29206.273, Time=2.65 sec
 ARIMA(1,1,1)(0,0,0)[0] intercept   : AIC=29200.842, Time=1.81 sec
 ARIMA(1,1,2)(0,0,0)[0] intercept   : AIC=29202.377, Time=2.59 sec
 ARIMA(0,1,2)(0,0,0)[0] intercept   : AIC=29206.042, Time=0.36 sec
 ARIMA(2,1,2)(0,0,0)[0] intercept   : AIC=29204.829, Time=2.68 sec
 ARIMA(1,1,1)(0,0,0)[0]             : AIC=29199.011, Time=0.56 sec
 ARIMA(0,1,1)(0,0,0)[0]             : AIC=29205.766, Time=0.10 sec
 ARIMA(1,1,0)(0,0,0)[0]             : AIC=29205.024, Time=0.11 sec
 ARIMA(2,1,1)(0,0,0

<class 'statsmodels.iolib.summary.Summary'>
"""
                               SARIMAX Results                                
==============================================================================
Dep. Variable:                      y   No. Observations:                 2603
Model:               SARIMAX(1, 1, 1)   Log Likelihood              -14596.505
Date:                Wed, 19 Oct 2022   AIC                          29199.011
Time:                        11:08:25   BIC                          29216.603
Sample:                    08-07-2015   HQIC                         29205.385
                         - 09-21-2022                                         
Covariance Type:                  opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ar.L1         -0.8044      0.037    -21.495      0.000      -0.878      -0.731
ma.L1          0.7528      0.043     17.505      0.000       0.669       0.837
sigma2      4373.9437     36.647    119.352      0.000    4302.116    4445.771
===================================================================================
Ljung-Box (L1) (Q):                   0.26   Jarque-Bera (JB):             59963.05
Prob(Q):                              0.61   Prob(JB):                         0.00
Heteroskedasticity (H):             101.79   Skew:                            -1.07
Prob(H) (two-sided):                  0.00   Kurtosis:                        26.42
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""